In [1]:
import os
import glob
import easymore
from pathlib import Path
from shutil import rmtree, copyfile
from datetime import datetime
from utils.read_files import make_default_path, read_from_control

In [2]:
# Store the name of the 'active' file in a variable
controlFile = 'control_EastRiver.txt'

In [3]:
# Catchment shapefile path & name
catchment_path = read_from_control(controlFile,'intersect_dem_path')
catchment_name = read_from_control(controlFile,'intersect_dem_name')

# Specify default path if needed
if catchment_path == 'default':
    catchment_path = make_default_path('shapefiles/catchment_intersection/with_dem', controlFile) # outputs a Path()
else:
    catchment_path = Path(catchment_path) # make sure a user-specified path is a Path()

In [4]:
# Forcing shapefile path & name
forcing_shape_path = read_from_control(controlFile,'forcing_shape_path')
forcing_shape_name = read_from_control(controlFile,'forcing_shape_name')

In [5]:
# Specify default path if needed
if forcing_shape_path == 'default':
    forcing_shape_path = make_default_path('shapefiles/forcing', controlFile) # outputs a Path()
else:
    forcing_shape_path = Path(forcing_shape_path) # make sure a user-specified path is a Path()

In [6]:
# Intersected shapefile path. Name is set by EASYMORE as [prefix]_intersected_shapefile.shp
intersect_path = read_from_control(controlFile,'intersect_forcing_path')
# Specify default path if needed
if intersect_path == 'default':
    intersect_path = make_default_path('shapefiles/catchment_intersection/with_forcing', controlFile) # outputs a Path()
else:
    intersect_path = Path(intersect_path) # make sure a user-specified path is a Path()

# Make the folder if it doesn't exist
intersect_path.mkdir(parents=True, exist_ok=True)

In [7]:
# Location of merged ERA5 files
forcing_merged_path = read_from_control(controlFile,'forcing_merged_path')
# Specify default path if needed
if forcing_merged_path == 'default':
    forcing_merged_path = make_default_path('forcing/2a_merged_data', controlFile) # outputs a Path()
else:
    forcing_merged_path = Path(forcing_merged_path) # make sure a user-specified path is a Path()

# Find files in folder
forcing_files = [forcing_merged_path/file for file in os.listdir(forcing_merged_path) if os.path.isfile(forcing_merged_path/file) and file.endswith('.nc')]
forcing_files.sort()

# Find where the temporary EASYMORE file needs to go

In [8]:
# Location for EASYMORE temporary storage
forcing_easymore_path = read_from_control(controlFile,'forcing_easymore_path')
# Specify default path if needed
if forcing_easymore_path == 'default':
    forcing_easymore_path = make_default_path('forcing/2b_tmp_easymore', controlFile) # outputs a Path()
else:
    forcing_easymore_path = Path(forcing_easymore_path) # make sure a user-specified path is a Path()

# Make the folder if it doesn't exist
forcing_easymore_path.mkdir(parents=True, exist_ok=True)

# Find where the area-weighted forcing needs to go

In [9]:
# Location for EASYMORE forcing output
forcing_basin_path = read_from_control(controlFile,'forcing_basin_avg_path')

# Specify default path if needed
if forcing_basin_path == 'default':
    forcing_basin_path = make_default_path('forcing/3_basin_averaged_data', controlFile) # outputs a Path()
else:
    forcing_basin_path = Path(forcing_basin_path) # make sure a user-specified path is a Path()

# Make the folder if it doesn't exist
forcing_basin_path.mkdir(parents=True, exist_ok=True)

# EASYMORE

In [10]:
# initialize easymore object
esmr = easymore.Easymore()

EASYMORE version 2.0.0 is initiated.


In [11]:
# add author name
esmr.author_name = "Daniel L. Hogan"

In [12]:
# Data license
esmr.license = "None"

In [13]:
# Case name, used in EASYMORE-generated file naes
esmr.case_name = read_from_control(controlFile,'domain_name')

In [14]:
#  ERA5 shapefile and variable names
# Variable names can be hardcoded because we set them when we generate this shapefile as part of the workflow
esmr.source_shp     = forcing_shape_path/forcing_shape_name # shapefile
esmr.source_shp_lat = read_from_control(controlFile,'forcing_shape_lat_name') # name of the latitude field
esmr.source_shp_lon = read_from_control(controlFile,'forcing_shape_lon_name') # name of the longitude field

In [15]:
# Catchment shapefile and variable names
esmr.target_shp = catchment_path/catchment_name
esmr.target_shp_ID  = read_from_control(controlFile,'catchment_shp_hruid') # name of the HRU ID field
esmr.target_shp_lat = read_from_control(controlFile,'catchment_shp_lat')   # name of the latitude field
esmr.target_shp_lon = read_from_control(controlFile,'catchment_shp_lon')   # name of the longitude field

In [16]:
# forcing file
esmr.source_nc = str(forcing_files[0]) # first file in the list; Path() to string
esmr.var_names = ['airpres',
                  'LWRadAtm',
                  'SWRadAtm',
                  'pptrate',
                  'airtemp',
                  'spechum',
                  'windspd'] # variable names of forcing data - hardcoded because we prescribe them during ERA5 merging
esmr.var_lat   = 'y'  # name of the latitude dimensions
esmr.var_lon   = 'x' # name of the longitude dimension
esmr.var_time  = 'time'      # name of the time dimension

# Temporary folder where the EASYMORE-generated GIS files and remapping file will be saved
esmr.temp_dir = str(forcing_easymore_path) + '/' # Path() to string; ensure the trailing '/' EASYMORE wants

# Output folder where the catchment-averaged forcing will be saved
esmr.output_dir = str(forcing_basin_path) + '/' # Path() to string; ensure the trailing '/' EASYMORE wants

In [17]:
# Netcdf settings
esmr.remapped_dim_id = 'hru'     # name of the non-time dimension; prescribed by SUMMA
esmr.remapped_var_id = 'hruId'   # name of the variable associated with the non-time dimension
esmr.format_list     = ['f4']    # variable type to save forcing as. Single entry here will be used for all variables
esmr.fill_value_list = ['-9999'] # fill value

In [18]:
# Flag that we do not want the data stored in .csv in addition to .nc
esmr.save_csv  = False
# Flag that we currently have no remapping file
esmr.remap_csv = ''  

In [19]:
# Enforce that we want our HRUs returned in the order we put them in
esmr.sort_ID = False

In [20]:
# Run EASYMORE
# Note on centroid warnings: in this case we use a regular lat/lon grid to represent ERA5 forcing and ...
#     centroid estimates without reprojecting are therefore acceptable.
# Note on deprecation warnings: this is a EASYMORE issue that cannot be resolved here. Does not affect current use.
esmr.nc_remapper()

EASYMORE is given multiple variables for remapping but only one format and fill value. EASYMORE repeats the format and fill value for all the variables in output files
EASYMORE will remap variable  airpres  from source file to variable  airpres  in remapped netCDF file
EASYMORE will remap variable  LWRadAtm  from source file to variable  LWRadAtm  in remapped netCDF file
EASYMORE will remap variable  SWRadAtm  from source file to variable  SWRadAtm  in remapped netCDF file
EASYMORE will remap variable  pptrate  from source file to variable  pptrate  in remapped netCDF file
EASYMORE will remap variable  airtemp  from source file to variable  airtemp  in remapped netCDF file
EASYMORE will remap variable  spechum  from source file to variable  spechum  in remapped netCDF file
EASYMORE will remap variable  windspd  from source file to variable  windspd  in remapped netCDF file
EASYMORE detects that the variables from the netCDF files are identicalin dimensions of the variables and latitude

/home/dlhogan/miniforge3/envs/easymore-env/lib/python3.9/site-packages/pyogrio/raw.py:723: RuntimeWarning: Value 206830746.981445819 of field HRU_area of feature 0 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
/home/dlhogan/miniforge3/envs/easymore-env/lib/python3.9/site-packages/pyogrio/raw.py:723: RuntimeWarning: Value 372989589.041010976 of field HRU_area of feature 1 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
/home/dlhogan/miniforge3/envs/easymore-env/lib/python3.9/site-packages/pyogrio/raw.py:723: RuntimeWarning: Value 163108286.000901699 of field HRU_area of feature 2 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
/home/dlhogan/miniforge3/envs/easymore-env/lib/python3.9/site-packages/pyogrio/raw.py:723: RuntimeWarning: Value 206830746.981445819 of field HRU_area of feature 0 not successfully written. Possibly due to

EASYMORE saved target shapefile for EASYMORE claculation as:
/storage/dlhogan/summa_modeling_data/domain_EastRiver/forcing/2b_tmp_easymore/EastRiver_target_shapefile.shp
EASYMORE detects case 1 - regular lat/lon
/storage/dlhogan/summa_modeling_data/domain_EastRiver/forcing/2b_tmp_easymore/EastRiver_source_shapefile.shp
EASYMORE created the shapefile from the netCDF file and saved it here:
EASMORE detects that target shapefile is inside the boundary of source netCDF file  and therefore correction for longitude values -180 to 180 or 0 to 360 is not performed even if  the correction_shp_lon flag is set to True [default is True]


/home/dlhogan/miniforge3/envs/easymore-env/lib/python3.9/site-packages/easymore/remapper.py:521: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_int.to_file(self.temp_dir+self.case_name+'_intersected_shapefile.shp') # save the intersected files
/home/dlhogan/miniforge3/envs/easymore-env/lib/python3.9/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'S_1_HRU_area' to 'S_1_HRU_ar'
  ogr_write(
/home/dlhogan/miniforge3/envs/easymore-env/lib/python3.9/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'S_1_center_lat' to 'S_1_center'
  ogr_write(
/home/dlhogan/miniforge3/envs/easymore-env/lib/python3.9/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'S_1_center_lon' to 'S_1_cent_1'
  ogr_write(
/home/dlhogan/miniforge3/envs/easymore-env/lib/python3.9/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: '

Ended at date and time 2025-03-07 15:43:52.170941
It took 0.90584 seconds to finish creating of the remapping file
---------------------------
------REMAPPING------
netcdf output file will be compressed at level 4
Remapping /storage/dlhogan/summa_modeling_data/domain_EastRiver/forcing/2a_merged_data/EastRiver_forcing_2022_2023.nc to /storage/dlhogan/summa_modeling_data/domain_EastRiver/forcing/3_basin_averaged_data/EastRiver_remapped_EastRiver_forcing_2022_2023.nc 
Started at date and time 2025-03-07 15:43:52.264328 
Ended at date and time 2025-03-07 15:44:08.551157 
It took 16.286829 seconds to finish the remapping of variable(s) 
---------------------
---------------------


In [21]:
# --- Move files to prescribed locations
# Remapping file 
remap_file = esmr.case_name + '_remapping.nc'
copyfile( esmr.temp_dir + remap_file, intersect_path / remap_file);

In [22]:
# Intersected shapefile
for file in glob.glob(esmr.temp_dir + esmr.case_name + '_intersected_shapefile.*'):
    copyfile( file, intersect_path / os.path.basename(file));
    
# Remove the temporary EASYMORE directory to save space
try:
    rmtree(esmr.temp_dir)
except OSError as e:
    print ("Error: %s - %s." % (e.filename, e.strerror))  